### 0. Import Libs

In [1]:
# kernel must be msd2
import pandas as pd
import geopandas as gpd
from shapely.geometry import *
import sys
from shapely import wkt
sys.path.append('../')
from util.epsg_transform import *
from util.distance import *
import constant as c
import os
import matplotlib.pyplot as plt

In [2]:
# Get df_bd
df_bd = pd.read_csv('../asset/preprocess/features/df_bd.csv', index_col=0, dtype={'mainpurpscd': str})
df_bd.dropna(subset=['x', 'y'], inplace=True)
x_5179, y_5179 = coord_tranformation(list(df_bd.x), list(df_bd.y), "epsg:4326", "epsg:5179")
df_bd['x_5179'] = x_5179
df_bd['y_5179'] = y_5179
df_bd['geometry'] = df_bd.apply(lambda row: Point([row['x_5179'], row['y_5179']]), axis = 1)
df_bd = gpd.GeoDataFrame(df_bd, geometry='geometry', crs = 'EPSG:5179')

In [3]:
# and for df_bd -> add gas, elc kwh info step by step
bdgas2020 = pd.read_csv('../asset/preprocess/features/df_bdgas_2020.csv')[['pnu', 'kwh']]
bdgas2021 = pd.read_csv('../asset/preprocess/features/df_bdgas_2021.csv')[['pnu', 'kwh']]
bdgas2022 = pd.read_csv('../asset/preprocess/features/df_bdgas_2022.csv')[['pnu', 'kwh']]
bdelct2020 = pd.read_csv('../asset/preprocess/features/df_bdelct_2020.csv')[['pnu', 'kwh']]
bdelct2021 = pd.read_csv('../asset/preprocess/features/df_bdelct_2021.csv')[['pnu', 'kwh']]
bdelct2022 = pd.read_csv('../asset/preprocess/features/df_bdelct_2022.csv')[['pnu', 'kwh']]

## gas2020
df_bd = df_bd.merge(bdgas2020, on='pnu', how='left'); df_bd = df_bd.rename(columns={'kwh': '2020gas'})
df_bd['2020gas'] = df_bd['2020gas'].fillna(0)

## gas2021
df_bd = df_bd.merge(bdgas2021, on='pnu', how='left'); df_bd = df_bd.rename(columns={'kwh': '2021gas'})
df_bd['2021gas'] = df_bd['2021gas'].fillna(0)

## gas2022
df_bd = df_bd.merge(bdgas2022, on='pnu', how='left'); df_bd = df_bd.rename(columns={'kwh': '2022gas'})
df_bd['2022gas'] = df_bd['2022gas'].fillna(0)

## elct2020
df_bd = df_bd.merge(bdelct2020, on='pnu', how='left'); df_bd = df_bd.rename(columns={'kwh': '2020elct'})
df_bd['2020elct'] = df_bd['2020elct'].fillna(0)

## gas2021
df_bd = df_bd.merge(bdelct2021, on='pnu', how='left'); df_bd = df_bd.rename(columns={'kwh': '2021elct'})
df_bd['2021elct'] = df_bd['2021elct'].fillna(0)

## gas2022
df_bd = df_bd.merge(bdelct2022, on='pnu', how='left'); df_bd = df_bd.rename(columns={'kwh': '2022elct'})
df_bd['2022elct'] = df_bd['2022elct'].fillna(0)

In [4]:
# Get df_grid
df_grid = pd.read_csv('../asset/preprocess/grid/df_grid.csv', index_col = 0)
df_grid = df_grid.rename(columns = {'geometry': 'geometry_grids'})
df_grid['geometry_grids'] = df_grid['geometry_grids'].apply(wkt.loads)
df_grid = df_grid[['idx', 'ADM_SECT_C', 'geometry_grids']]
df_grid = df_grid.rename(columns={'idx': 'grid_idx'})
df_grid = gpd.GeoDataFrame(df_grid, crs = 'EPSG:5179', geometry = 'geometry_grids')
df_grid = df_grid.to_crs({'init':'epsg:5179'})
df_grid['grid_idx'] = df_grid['grid_idx'].astype(str)

# For given bd info(coord x,y) find grid that x, y falles in and assign
df_bd_assigned = gpd.sjoin(df_bd, df_grid, op = 'within', how = 'left')
df_bd_assigned = df_bd_assigned[~df_bd_assigned['grid_idx'].isna()]
df_bd_assigned = df_bd_assigned.drop('index_right', axis=1)
df_bd_assigned['grid_idx'] = df_bd_assigned['grid_idx'].astype(float).astype(int).astype(str)
df_bd_assigned['ADM_SECT_C'] = df_bd_assigned['ADM_SECT_C'].astype(float).astype(int).astype(str)

# Concat dfs
df_grid_feature = df_bd_assigned.merge(df_grid, on = 'grid_idx')
df_grid_feature = df_grid_feature.drop('ADM_SECT_C_y', axis=1)
df_grid_feature = df_grid_feature.rename(columns = {'ADM_SECT_C_x': 'ADM_SECT_C'})

gdf_cbds = gpd.GeoDataFrame(df_grid_feature, geometry = 'geometry_grids')
gdf_cbds['ADM_SECT_C'] = gdf_cbds['ADM_SECT_C'].astype(float).astype(int).astype(str) # 11110.0 -> '11110'
ref_col = ['ADM_SECT_C', 'idx']
geo_col = ['geometry_grids']
var_col = ['platarea', 'archarea', 'bcrat',\
       'totarea', 'vlrat', 'heit', 'grndflrcnt', 'ugrndflrcnt',\
       'rideuseelvtcnt', 'emgenuseelvtcnt', 'useaprday']
y_col = ['2019', '2020', '2021', '2022']

In [5]:
# Append 'population', 'od' data for each grids
## get grids data
grids_ = gpd.GeoDataFrame(df_grid, geometry='geometry_grids')
grids_['center'] = grids_['geometry_grids'].centroid
grids_ = grids_[['grid_idx', 'center']]

## and get population data
df_popse = pd.read_csv('../asset/preprocess/features/df_popse.csv', index_col = 0)
df_popse_= df_popse[['202001', '202101', '202201', 'x_5179', 'y_5179']]
df_popse_['202001'] = df_popse_['202001'].fillna(0)
df_popse_['202101'] = df_popse_['202101'].fillna(0)
df_popse_['202201'] = df_popse_['202201'].fillna(0)
df_popse_ = gpd.GeoDataFrame(df_popse_, geometry=gpd.points_from_xy(df_popse_['x_5179'], df_popse_['y_5179']))


In [6]:
## and get metro data
df_metro = pd.read_csv('../asset/preprocess/features/df_metro.csv', index_col = 0)
df_metro['od'] = df_metro.loc[:, '0405O':'0304D'].sum(axis=1)
df_metro = df_metro[df_metro['호선명'].isin(c.SEOUL_METRO_NMS)] # confine 호선명 to 1호선 ~ 9호선
df_metro_ = df_metro[['호선명', '지하철역', 'year', 'od', 'x', 'y', 'x_5179', 'y_5179']]
df_metro_ = gpd.GeoDataFrame(df_metro_, geometry=gpd.points_from_xy(df_metro_['x_5179'], df_metro_['y_5179']))
df_metro_stations = df_metro_[['호선명', '지하철역', 'x', 'y', 'x_5179', 'y_5179', 'geometry']]
df_metro_stations = df_metro_stations.drop_duplicates()
od2020s = []
od2021s = []
od2022s = []
for i in range(len(df_metro_stations)):
    hosun = df_metro_stations.iloc[i]['호선명']
    station_name = df_metro_stations.iloc[i]['지하철역']
    ods = []
    for year in [2020, 2021, 2022]:
        cond1 = df_metro_['year'] == year 
        cond2 = df_metro_['호선명'] == hosun
        cond3 = df_metro_['지하철역'] == station_name
        try:
            od_that_year = df_metro_[cond1 & cond2 & cond3]['od'].values[0]
        except:
            od_that_year = 0
        ods.append(od_that_year)
    od2020s.append(ods[0])
    od2021s.append(ods[1])
    od2022s.append(ods[2])
df_metro_stations['2020'] = od2020s
df_metro_stations['2021'] = od2021s
df_metro_stations['2022'] = od2022s

# deduct od value below 1000
df_metro_stations = df_metro_stations[df_metro_stations['2020'] >= 1000] 
df_metro_stations = df_metro_stations[df_metro_stations['2021'] >= 1000]
df_metro_stations = df_metro_stations[df_metro_stations['2022'] >= 1000]

In [7]:
# applying distance decay for x, y coordinate of building
# distance decay for the effect of distance on spatial interaction
# we sum the product of value(population) and I(distance decay index)
# calculating I = e ^ (-d) where d is distance in km -> ex. distance 100m, population 500 -> 500 * e^(-0.1) = 500 * 0.9048374180359595
# distance threshold
DISTANCE_THRES = 3000  # in meters

# grid and population
ppop2020s = [] # abbr for processed population
ppop2021s = []
ppop2022s = [] 

for i in range(len(grids_)): # i for grid
# for i in range(20): # i for grid
    ppop2020, ppop2021, ppop2022 = 0, 0, 0
    for j in range(len(df_popse_)): # j for popse
        meter_distance = grids_['center'].iloc[i].distance(df_popse_['geometry'].iloc[j])
        if meter_distance <= DISTANCE_THRES:
            pop2020 = df_popse_['202001'].iloc[j]
            ppop2020 += distance_decay(pop2020, meter_distance)
            pop2021 = df_popse_['202101'].iloc[j]
            ppop2021 += distance_decay(pop2021, meter_distance)
            pop2022 = df_popse_['202201'].iloc[j]
            ppop2022 += distance_decay(pop2022, meter_distance)
        else:
            pass
    ppop2020s.append(ppop2020)
    ppop2021s.append(ppop2021)
    ppop2022s.append(ppop2022)

grids_['pop2020'] = ppop2020s
grids_['pop2021'] = ppop2021s
grids_['pop2022'] = ppop2022s

In [8]:
# grid and metro
pmetro2020s = [] # abbr for processed metro
pmetro2021s = []
pmetro2022s = [] 

for i in range(len(grids_)): # i for grid
# for i in range(20): # i for grid
    pmetro2020, pmetro2021, pmetro2022 = 0, 0, 0
    for j in range(len(df_metro_stations)): # j for metro
        meter_distance = grids_['center'].iloc[i].distance(df_metro_stations['geometry'].iloc[j])
        if meter_distance <= DISTANCE_THRES:
            metro2020 = df_metro_stations['2020'].iloc[j]
            pmetro2020 += distance_decay(metro2020, meter_distance)
            metro2021 = df_metro_stations['2021'].iloc[j]
            pmetro2021 += distance_decay(metro2021, meter_distance)
            metro2022 = df_metro_stations['2022'].iloc[j]
            pmetro2022 += distance_decay(metro2022, meter_distance)
        else:
            pass
    pmetro2020s.append(pmetro2020)
    pmetro2021s.append(pmetro2021)
    pmetro2022s.append(pmetro2022)

grids_['od2020'] = pmetro2020s
grids_['od2021'] = pmetro2021s
grids_['od2022'] = pmetro2022s

In [9]:
grids_.to_csv('../asset/preprocess/grid/df_grid_pop_od.csv')

In [10]:
gdf_scbd = gdf_cbds[gdf_cbds['ADM_SECT_C'].isin(c.SCBD_CDS)]
gdf_gbd = gdf_cbds[gdf_cbds['ADM_SECT_C'].isin(c.GBD_CDS)]
gdf_ybd = gdf_cbds[gdf_cbds['ADM_SECT_C'] == c.YDPG_CD]

In [11]:
gdf_ybd['useaprday']

10527    19981205.0
10528    19931206.0
10529    20100531.0
10530    19941123.0
10531    20191206.0
            ...    
12224    19960817.0
12225           NaN
12226    20051125.0
12227    20031229.0
12228    20020822.0
Name: useaprday, Length: 1702, dtype: float64

### 2. Get Seoul Boundary

In [12]:
seoul_shapefile = os.path.join(c.SSD_PATH, 'nsdi/LARD_ADM_SECT_SGG_서울/LARD_ADM_SECT_SGG_11.shp')
seoul = gpd.read_file(seoul_shapefile)
seoul = seoul[['ADM_SECT_C', 'geometry']]
scbd_boundary = seoul[seoul['ADM_SECT_C'].isin(c.SCBD_CDS)]
gbd_boundary = seoul[seoul['ADM_SECT_C'].isin(c.GBD_CDS)]
ybd_boundary = seoul[seoul['ADM_SECT_C'] == c.YDPG_CD]

### 3. Save dfs

In [13]:
gdf_cbds.to_csv('../asset/preprocess/grid/gdf_cbds.csv')
gdf_cbds.head(20).to_csv('../asset/preprocess/grid/gdf_cbds_head.csv')
scbd_boundary.to_csv('../asset/preprocess/grid/scbd_boundary.csv')
gbd_boundary.to_csv('../asset/preprocess/grid/gbd_boundary.csv')
ybd_boundary.to_csv('../asset/preprocess/grid/ybd_boundary.csv')
gdf_cbds.head(5)

,pnu,rlgnm,rlgcd,sigungunm,lawdongnm,linm,bun,ji,naroadnm,naroadcd,...,geometry,2020gas,2021gas,2022gas,2020elct,2021elct,2022elct,grid_idx,ADM_SECT_C,geometry_grids
0,1168011800109590021,서울특별시,11,강남구,도곡동,NaN,959,21,남부순환로361길,116804166092,...,POINT (959090.778 1943027.542),17717.0,18652.0,19903.0,11829.0,11344.0,10110.0,36381,11680,"POLYGON ((959035.230 1942965.548, 959035.230 1..."
1,1168011800109590020,서울특별시,11,강남구,도곡동,NaN,959,20,남부순환로361길,116804166092,...,POINT (959074.038 1943032.991),20282.0,23407.0,23576.0,15926.0,18720.0,17161.0,36381,11680,"POLYGON ((959035.230 1942965.548, 959035.230 1..."
2,1168011800109200002,서울특별시,11,강남구,도곡동,NaN,920,2,남부순환로361길,116804166092,...,POINT (959117.186 1943027.622),30576.0,21170.0,18836.0,20915.0,18064.0,17210.0,36381,11680,"POLYGON ((959035.230 1942965.548, 959035.230 1..."
3,1168011800109190011,서울특별시,11,강남구,도곡동,NaN,919,11,남부순환로363길,116804166093,...,POINT (959128.097 1943046.156),3312.0,5056.0,4137.0,3772.0,3681.0,3967.0,36381,11680,"POLYGON ((959035.230 1942965.548, 959035.230 1..."
4,1168011800109590016,서울특별시,11,강남구,도곡동,NaN,959,16,남부순환로359길,116804166091,...,POINT (959036.318 1943065.615),3407.0,3906.0,7119.0,8802.0,8013.0,9328.0,36382,11680,"POLYGON ((959035.230 1943065.548, 959035.230 1..."


In [14]:
# Read gdf_cbds
gdf_cbds = pd.read_csv('../asset/preprocess/grid/gdf_cbds.csv', index_col = 0)
gdf_cbds.head(5)

,pnu,rlgnm,rlgcd,sigungunm,lawdongnm,linm,bun,ji,naroadnm,naroadcd,...,geometry,2020gas,2021gas,2022gas,2020elct,2021elct,2022elct,grid_idx,ADM_SECT_C,geometry_grids
0,1168011800109590021,서울특별시,11,강남구,도곡동,NaN,959,21,남부순환로361길,116804166092,...,POINT (959090.7780094582 1943027.54165114),17717.0,18652.0,19903.0,11829.0,11344.0,10110.0,36381,11680,"POLYGON ((959035.230213883 1942965.54807064, 9..."
1,1168011800109590020,서울특별시,11,강남구,도곡동,NaN,959,20,남부순환로361길,116804166092,...,POINT (959074.0375926002 1943032.991403951),20282.0,23407.0,23576.0,15926.0,18720.0,17161.0,36381,11680,"POLYGON ((959035.230213883 1942965.54807064, 9..."
2,1168011800109200002,서울특별시,11,강남구,도곡동,NaN,920,2,남부순환로361길,116804166092,...,POINT (959117.1861895586 1943027.621584438),30576.0,21170.0,18836.0,20915.0,18064.0,17210.0,36381,11680,"POLYGON ((959035.230213883 1942965.54807064, 9..."
3,1168011800109190011,서울특별시,11,강남구,도곡동,NaN,919,11,남부순환로363길,116804166093,...,POINT (959128.096725813 1943046.15622537),3312.0,5056.0,4137.0,3772.0,3681.0,3967.0,36381,11680,"POLYGON ((959035.230213883 1942965.54807064, 9..."
4,1168011800109590016,서울특별시,11,강남구,도곡동,NaN,959,16,남부순환로359길,116804166091,...,POINT (959036.3184876485 1943065.615017564),3407.0,3906.0,7119.0,8802.0,8013.0,9328.0,36382,11680,"POLYGON ((959035.230213883 1943065.54807064, 9..."


In [15]:
# Read {}_boundary
ybd_boundary = pd.read_csv('../asset/preprocess/grid/ybd_boundary.csv', index_col = 0)
ybd_boundary.head(10)

,ADM_SECT_C,geometry
6,11560,"POLYGON ((945280.001025055 1950949.31881145, 9..."
